<a href="https://colab.research.google.com/github/cibalcazarg/cibalcazar/blob/main/Mapa_interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium geopandas

In [218]:
import folium
import geopandas as gpd
from folium import Choropleth, GeoJson
from folium import FeatureGroup

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving plantas.txt to plantas.txt


In [309]:
cantones = gpd.read_file('/content/Loja.shp')  # Reemplaza con tu archivo

In [310]:
MyMap = folium.Map(location=[-4.0027, -79.2066], tiles='OpenStreetMap', zoom_start=10, max_zoom=19, control_scale=True)

In [311]:
import pandas as pd
plants = pd.read_csv('plantas.txt', delimiter=';')

In [312]:
print(plants.head())

   OBJECTID                          Name        cientifico  \
0         1   Dodson, Calaway Homer - 645  Iresine  diffusa   
1         2   Madsen, Jens Elgaard - 7068  Iresine  diffusa   
2         3   Madsen, Jens Elgaard - 7068  Iresine  diffusa   
3         4  Dodson, Calaway Homer - 1499  Iresine  diffusa   
4         5         Ellemann, Lis - 66516  Iresine  diffusa   

               comun              POINT_X             POINT_Y  \
0  Chulku,  escancel  -79,249999999999943  -3,779999999999973   
1  Chulku,  escancel  -79,979999999999961  -3,999999999999943   
2  Chulku,  escancel  -79,979999999999961  -3,999999999999943   
3  Chulku,  escancel  -79,219999999999970  -3,999999999999943   
4  Chulku,  escancel  -79,149999999999977  -3,919999999999959   

             POINT_Z  POINT_M  
0  0,000000000000000      NaN  
1  0,000000000000000      NaN  
2  0,000000000000000      NaN  
3  0,000000000000000      NaN  
4  0,000000000000000      NaN  


In [313]:
plants['POINT_X'] = plants['POINT_X'].str.replace(',', '.').astype(float)
plants['POINT_Y'] = plants['POINT_Y'].str.replace(',', '.').astype(float)

In [314]:
MyMap = folium.Map(location=[plants['POINT_Y'].mean(), plants['POINT_X'].mean()], zoom_start=10)

In [315]:
folium.GeoJson(
    cantones,
    name='Límites de Cantones',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.1,
    }
).add_to(MyMap)

In [305]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
species = plants['comun'].unique()
color_map = {species[i]: colors[i % len(colors)] for i in range(len(species))}

In [316]:
# Agregar marcadores al mapa
for especie in species:
    specie_data = plants[plants['comun'] == especie]

    # Depuración: Imprimir número de registros por especie
    print(f"Especie: {especie}, Número de registros: {len(specie_data)}")

    marker_layer = FeatureGroup(name=especie)

    for idx, row in specie_data.iterrows():
        if pd.notnull(row['POINT_Y']) and pd.notnull(row['POINT_X']):
            folium.Marker(
                location=[row['POINT_Y'], row['POINT_X']],
                popup=folium.Popup(f"""
                    <div style="font-size: 14px;">
                        <b>Nombre común:</b> {row['comun']}<br>
                        <b>Nombre científico:</b> {row['cientifico']}<br>
                        <b>Altitud:</b> {row.get('altitud', 'N/A')}
                    </div>
                """, max_width=300),
                icon=folium.Icon(color=color_map.get(especie, 'blue'))  # Default to 'blue' if color not found
            ).add_to(marker_layer)

    marker_layer.add_to(MyMap)

Especie: Chulku,  escancel, Número de registros: 8
Especie: Anis, Número de registros: 5
Especie: Moradilla, Número de registros: 21
Especie: Eneldo,  hinojo, Número de registros: 1
Especie: tigrecillo, Número de registros: 2


In [317]:
# Agregar control de capas al mapa
folium.LayerControl().add_to(MyMap)

In [318]:
legend_html = '''
     <div style="position: fixed;
     bottom: 70px; left: 10px; width: 200px; height: 200px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     &nbsp; <b> Leyenda </b> <br>
     {}
     </div>
'''.format(''.join([f'&nbsp; <i class="fa fa-map-marker fa-2x" style="color:{color_map[specie]}"></i>&nbsp;{specie}<br>' for specie in species]))

MyMap.get_root().html.add_child(folium.Element(legend_html))

# Agregar título en la parte superior
titulo_html = '''
    <div style="position: absolute; top: 14px; left: 14px; background-color: white;
                border: 2px solid black; padding: 5px; border-radius: 5px; z-index: 1000;">
        <h2 style="font-weight: bold;">Mapa de ubicación de montes de horchata</h2>
    </div>
'''
MyMap.get_root().html.add_child(folium.Element(titulo_html))

# Agregar datos en la parte inferior
datos_html = '''
    <div style="position: absolute; bottom: 10px; left: 10px; background-color: white;
                border: 2px solid black; padding: 5px; border-radius: 2px; z-index: 1000;">
        <p>La horchata Lojana como patrimonio inmaterial</p>
    </div>
'''
MyMap.get_root().html.add_child(folium.Element(datos_html))

In [319]:
MyMap

In [321]:
# Guardar el mapa en un archivo HTML
MyMap.save('/content/mapa_plantas.html')

# Abrir el archivo HTML en Google Colab
from IPython.display import IFrame
IFrame('/content/mapa_plantas.html', width=800, height=600)